In [ ]:
!pip install sionna

In [ ]:
import tensorflow as tf
from tensorflow.keras import Model
import numpy as np
import sionna as sn
import matplotlib.pyplot as plt 

Visualize Constellation

In [ ]:
const = sn.mapping.Constellation("pam", num_bits_per_symbol=2)
const.show()

Simulate 5G LDPC Code over AWGN Channel

In [ ]:
class LDPC_4PAM_BICM(Model):
    def __init__(self, num_bits_per_symbol, n, coderate, num_iterations, batch_size):
        super().__init__()

        self.num_bits_per_symbol = num_bits_per_symbol
        self.n = n
        self.k = int(n*coderate)
        self.coderate = coderate
        self.num_iterations = num_iterations
        self.batch_size = batch_size
        self.constellation = sn.mapping.Constellation("pam", self.num_bits_per_symbol)

        self.interleaver = sn.fec.interleaving.RandomInterleaver(dtype=tf.float32, axis=1)
        self.deinterleaver = sn.fec.interleaving.Deinterleaver(self.interleaver, dtype=tf.float32)

        self.mapper = sn.mapping.Mapper(constellation=self.constellation)
        self.demapper = sn.mapping.Demapper("app", constellation=self.constellation, with_prior=True)

        self.binary_source = sn.utils.BinarySource()
        self.awgn_channel = sn.channel.AWGN()

        self.encoder = sn.fec.ldpc.LDPC5GEncoder(self.k, self.n)
        self.soft_decoder = sn.fec.ldpc.LDPC5GDecoder(self.encoder, hard_out=False, return_infobits=False)
        self.hard_decoder = sn.fec.ldpc.LDPC5GDecoder(self.encoder, hard_out=True, return_infobits=True)

    # @tf.function
    def __call__(self, batch_size, ebno_db):

        # Get N0 for current Eb/N0 value
        no = sn.utils.ebnodb2no(ebno_db, num_bits_per_symbol=self.num_bits_per_symbol, coderate=self.coderate)

        # Encoding
        bits = self.binary_source([batch_size, self.k])
        codewords = self.encoder(bits)
        interleaved_codewords = self.interleaver(codewords)
        x = self.mapper(interleaved_codewords)

        # Channel
        y = self.awgn_channel([x, no])

        # Decoding
        demapper_priors = tf.zeros([self.batch_size, self.n//2, self.num_bits_per_symbol])
        demapper_llrs = self.demapper([y, demapper_priors, no])

        for idxit in range(self.num_iterations):

          # Subtract priors from demapper output
          demapper_extrinsic_llrs = demapper_llrs - tf.reshape(demapper_priors, [self.batch_size, self.n])

          # Deinterleave demapper extrinsic information
          decoder_priors = tf.clip_by_value(self.deinterleaver(demapper_extrinsic_llrs), -20, 20)

          # Run SISO LDPC decoder
          decoder_llrs = self.soft_decoder(decoder_priors)

          # Subtract priors from decoder output
          decoder_extrinsic_llrs = decoder_llrs - decoder_priors

          # Interleave decoder extrinsic information
          demapper_priors = self.interleaver(decoder_extrinsic_llrs)

          # Demap channel outputs with new prior information
          demapper_priors = tf.reshape(demapper_priors, [self.batch_size, self.n//2, self.num_bits_per_symbol])
          demapper_llrs = self.demapper([y, demapper_priors, no])

        # Deinterleave channel outputs
        deinterleaved_llrs = self.deinterleaver(demapper_llrs)

        # Pass through hard decision decoder
        bits_hat = self.hard_decoder(deinterleaved_llrs)

        # Return results
        return bits, bits_hat

In [ ]:
# Modulation Parameters
bits_per_symbol = 2

# Code Parameters
k = 5888
n = 7350*bits_per_symbol
r = k/n

# Simulation Parameters
snrs_to_test = np.array([1.0, 1.25, 1.5, 1.75, 2.0, 2.25, 2.5, 2.75])
batch_size = 2000
target_block_errors = 750
max_mc_iterations = 50
num_iterations = 3

# Define Simulations
ldpc_4_pam_bicm = LDPC_4PAM_BICM(num_bits_per_symbol=bits_per_symbol, n=n, coderate=r, num_iterations=num_iterations, batch_size=batch_size)

In [ ]:
ber_plots = sn.utils.PlotBER("4-PAM NR LDPC BICM")
ber_plots.simulate(ldpc_4_pam_bicm,
                   ebno_dbs=snrs_to_test,
                   batch_size=batch_size,
                   num_target_block_errors=target_block_errors,
                   legend="4-PAM NR LDPC BICM",
                   soft_estimates=False,
                   max_mc_iter=max_mc_iterations,
                   show_fig=False,
                   forward_keyboard_interrupt=False);